In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
# Read in the webpage into a Dataframe
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

In [5]:
# Ignore cells with a borough that is Not assigned.
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

In [6]:
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [7]:
# Cmbine cases where neighbourhood exists in more than one Postal area
df1=df.groupby("Postcode").agg(lambda x:','.join(set(x)))
df1.loc[df1['Neighbourhood']=="Not assigned",'Neighbourhood']=df1.loc[df1['Neighbourhood']=="Not assigned",'Borough']
df1.shape

(103, 2)

In [7]:
df1.head(10)

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Morningside,Guildwood,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park"
M1L,Scarborough,"Golden Mile,Clairlea,Oakridge"
M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest"


In [12]:
# Download the csv file
!wget -q -O 'Geo.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

# Read it into a dataframe
geo_df = pd.read_csv('Geo.csv')

# Assign new columns to the df since postcode is spelled differently
geo_df.columns = ['Postcode', 'Latitude','Longitude']

geo_df.head()

Data downloaded!


,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
# Merge the two dataframes joining on Postcode
df_merge = pd.merge(df1, geo_df, on=['Postcode'], how='inner')

neighborhoods = df_merge[['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']].copy()
neighborhoods.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
